# Data integration - OpenCivitas

In questo IPython Notebook è presente la procedura utilizzata per integrare i dati OpenCivitas con info dei comuni. L'output della procedura sono una serie di file, che vedono l'aggiunta di alcune variabili rispetto i dati iniziali. Nel complesso, l'informazione contenuta nei file è la seguente:


| Nome | Tipo   | Descrizione                     |
|------|--------|---------------------------------|
| ANNO | number | Anno di analisi  |
| Codice_Nazionale | string  | codice catastale del comune |
| Codice_Istat | string  | codice ISTAT |
| Denominazione Italiana | string  | Nome comune |
| Provincia | string  | Nome capoluogo di provincia cui appartiene il comune |
| Sigla Provincia | string  | Sigla capoluogo di provincia cui appartiene il comune |
| Regione | string  | Regione cui appartiene il comune |
| AreaGeo | string  | Area geografica cui appartiene il comune |
| ... | ... | ...|
| ... | ... | ...|
| ... | ... | ...|
| __Variabili in fabbisogni__|||
| ... | ... | ...|
| ... | ... | ...|
| ... | ... | ...|

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os

* Carichiamo la lista dei fali che dobbiamo modificare

In [2]:
# Importare lista dati
lista_file = os.listdir("data/")

* Per ogni file compiamo le seguenti operazioni (commenti nel codice):

    * Per ogni anno, estraiamo le righe del dataset corrispondenti
    * Carichiamo il dataset che contiene l'elenco di tutti comuni
    * Facciamo la `join` tra il dataset dei fabbisogni e quello dei comuni
 
Per aggiungere altre possibili informazioni utili :
* Denominazione Italiana (nome comune)
* Provincia (cui il comune appartiene) 
* Sigla Provincia (cui il comune appartiene) 
* Regione (cui il comune appartiene)
* AreaGeo (area geografica)

Aggiungo una join con un dataset ISTAT. 

__A seguire__: una descrizione più dei risultati intermedi ad ogni passaggio per un unico dataset su un anno specifico.

# Esempio con uno specifico dataset di fabbisogni e anno

In [3]:
df_fab = pd.read_csv('data/FC01A_serv_entrate.csv', sep = ';')
anno = 2010

* Estraiamo le righe del dataset corrispondenti all'anno

In [4]:
df_fab_anno = df_fab[df_fab["ANNO"] == anno]
df_fab_anno.head(3)

,ANNO,COMUNE_CAT_COD,CAT_DET_COD_1,CAT_DET_COD_2,CAT_DET_COD_3,CAT_DET_COD_4,CAT_DET_COD_5,COEFF_TEORICO,IND1,IND3,...,IND9,IND10,IND11,LQP_COD_1,LQP_COD_2,LQP_COD_3,LQP_COD_4,LQP_COD_5,LQP_COD_6,LQP_COD_S
0,2010,A052,0.007314,0.235773,0.135332,0.164862,0.456719,0.000447,24.199183,123.729184,...,9.535919,28.0,66.637397,123.729184,-1.039833,0,0,10.0,5.0,3.4
1,2010,A146,0.234971,0.059315,0.136848,0.268198,0.300667,0.000019,0.000000,NaN,...,0.000000,3.0,51.259253,NaN,NaN,1,0,NaN,NaN,NaN
2,2010,A182,0.001551,0.189163,0.224143,0.137365,0.447778,0.002147,5.483805,-51.217919,...,20.745082,20.0,53.598403,-51.217919,-12.525939,0,0,3.0,3.0,5.0


In [5]:
print ('Numero comuni anno '+ str(anno) +':', len(df_fab_anno['COMUNE_CAT_COD'].unique()))

Numero comuni anno 2010: 6702


* Carichiamo il dataset che contiene l'elenco di tutti comuni

In [6]:
# Carico il file 
df_comuni = pd.read_csv('ElencoComuniAttuali_20170918.csv', sep = ';')
# Trattengo le variabili di interesse
df_comuni = df_comuni[['Codice Nazionale','Codice Istat','Denominazione Italiana', 'Sigla Provincia']] 
# E le rinomino per poter fare la join con il dataset dei fabbisogni
df_comuni.columns = ['COMUNE_CAT_COD','ISTAT','Denominazione Italiana', 'Sigla Provincia']

In [7]:
df_comuni.head(3)

,COMUNE_CAT_COD,ISTAT,Denominazione Italiana,Sigla Provincia
0,A001,28001.0,ABANO TERME,PD
1,A004,98001.0,ABBADIA CERRETO,LO
2,A005,97001.0,ABBADIA LARIANA,LC


* Facciamo la join tra il dataset dei fabbisogni e quello dei comuni

In [8]:
# Left join tra i fabbisogni ed i dati dei comuni
df_comuni_cod = pd.merge(df_fab_anno,df_comuni, on = 'COMUNE_CAT_COD',how = "left")

df_comuni_cod = df_comuni_cod[['COMUNE_CAT_COD','ISTAT', 'Denominazione Italiana',
   'Sigla Provincia','ANNO', 'CAT_DET_COD_1', 'CAT_DET_COD_2',
   'CAT_DET_COD_3', 'CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO',
   'IND1', 'IND3', 'IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10',
   'LQP_COD_1', 'LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5',
   'LQP_COD_6', 'LQP_COD_S']]

In [9]:
df_comuni_cod.head(3)

,COMUNE_CAT_COD,ISTAT,Denominazione Italiana,Sigla Provincia,ANNO,CAT_DET_COD_1,CAT_DET_COD_2,CAT_DET_COD_3,CAT_DET_COD_4,CAT_DET_COD_5,...,IND8,IND9,IND10,LQP_COD_1,LQP_COD_2,LQP_COD_3,LQP_COD_4,LQP_COD_5,LQP_COD_6,LQP_COD_S
0,A052,6001.0,ACQUI TERME,AL,2010,0.007314,0.235773,0.135332,0.164862,0.456719,...,0.0,9.535919,28.0,123.729184,-1.039833,0,0,10.0,5.0,3.4
1,A146,6002.0,ALBERA LIGURE,AL,2010,0.234971,0.059315,0.136848,0.268198,0.300667,...,0.0,0.000000,3.0,NaN,NaN,1,0,NaN,NaN,NaN
2,A182,6003.0,ALESSANDRIA,AL,2010,0.001551,0.189163,0.224143,0.137365,0.447778,...,0.0,20.745082,20.0,-51.217919,-12.525939,0,0,3.0,3.0,5.0


In [10]:
print ('SANITY CHECK 1: Numero di comuni dopo la join: ', len(df_comuni_cod))

SANITY CHECK 1: Numero di comuni dopo la join:  6702


* Carico il seguente dataset ISTAT

In [11]:
# Carica il file che racchiude informazioni extra sui comuni (provincia, regione, area geografica)
df_comuni_info = pd.read_csv('info_comuni.txt', sep = ";")
df_comuni_info.head(3)

,Comune,ISTAT,Provincia,SiglaProv,Regione,AreaGeo,PopResidente,PopStraniera,DensitaDemografica,SuperficieKmq,...,AltezzaMassima,ZonaAltimetrica,TipoComune,GradoUrbaniz,IndiceMontanita,ZonaClimatica,ZonaSismica,ClasseComune,Latitudine,Longitudine
0,Abano Terme,28001,Padova,PD,Veneto,Nord-Est,20002,2001,"931,89","21,408",...,80,Montagna Interna,No capoluogo,Elevato,Non montano,E,4,Polo di attrazione intercomunale,"45,3594444444444","11,7894444444444"
1,Abbadia Cerreto,98001,Lodi,LO,Lombardia,Nord-Ovest,284,13,"46,62","6,199",...,70,Pianura,No capoluogo,Medio,Non montano,E,4,Area di cintura,"45,3122222222222","9,59277777777778"
2,Abbadia Lariana,97001,Lecco,LC,Lombardia,Nord-Ovest,3209,135,"191,93","16,673",...,1700,Montagna Interna,No capoluogo,Medio,Totalmente montano,E,4,Area periferica,"45,8991666666667","9,33361111111111"


* Effettua la join con `df_comuni_cod`

In [12]:
# Effettua la join con df_comuni_cod 
df_info_comuni_fabbisogni = pd.merge(df_comuni_cod, df_comuni_info, on = 'ISTAT',how = "left")
df_info_comuni_fabbisogni.head(3)

,COMUNE_CAT_COD,ISTAT,Denominazione Italiana,Sigla Provincia,ANNO,CAT_DET_COD_1,CAT_DET_COD_2,CAT_DET_COD_3,CAT_DET_COD_4,CAT_DET_COD_5,...,AltezzaMassima,ZonaAltimetrica,TipoComune,GradoUrbaniz,IndiceMontanita,ZonaClimatica,ZonaSismica,ClasseComune,Latitudine,Longitudine
0,A052,6001,ACQUI TERME,AL,2010,0.007314,0.235773,0.135332,0.164862,0.456719,...,453.0,Collina Interna,No capoluogo,Basso,Non montano,E,3,Polo di attrazione urbana,"44,6763888888889","8,465"
1,A146,6002,ALBERA LIGURE,AL,2010,0.234971,0.059315,0.136848,0.268198,0.300667,...,1559.0,Montagna Interna,No capoluogo,Basso,Totalmente montano,E,3,Area intermedia,"44,7005555555556","9,06694444444444"
2,A182,6003,ALESSANDRIA,AL,2010,0.001551,0.189163,0.224143,0.137365,0.447778,...,268.0,Pianura,Capoluogo di Provincia,Medio,Non montano,E,3,Polo di attrazione urbana,"44,9125","8,61444444444444"


* Rinomina le variabili e prendi quelle di interesse

In [13]:
# Rinomina variabili
df_info_comuni_fabbisogni.columns = ['Codice_Nazionale', 'Codice_Istat', 'Denominazione Italiana', 'Sigla Provincia',
   'ANNO', 'CAT_DET_COD_1', 'CAT_DET_COD_2', 'CAT_DET_COD_3',
   'CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO', 'IND1', 'IND3',
   'IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10', 'LQP_COD_1',
   'LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5', 'LQP_COD_6',
   'LQP_COD_S', 'Comune', 'Provincia', 'SiglaProv', 'Regione', 'AreaGeo',
   'PopResidente', 'PopStraniera', 'DensitaDemografica', 'SuperficieKmq',
   'AltezzaCentro', 'AltezzaMinima', 'AltezzaMassima', 'ZonaAltimetrica',
   'TipoComune', 'GradoUrbaniz', 'IndiceMontanita', 'ZonaClimatica',
   'ZonaSismica', 'ClasseComune', 'Latitudine', 'Longitudine']

# Seleziona le variabili di interesse
df_info_comuni_fabbisogni = df_info_comuni_fabbisogni[['ANNO','Codice_Nazionale', 'Codice_Istat',
                                                   'Denominazione Italiana',
                                                   'Provincia','Sigla Provincia',
                                                   'Regione', 'AreaGeo',
                                                   'CAT_DET_COD_1', 'CAT_DET_COD_2', 'CAT_DET_COD_3','CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO',
                                                   'IND1', 'IND3','IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10', 
                                                   'LQP_COD_1','LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5', 'LQP_COD_6','LQP_COD_S']]

df_info_comuni_fabbisogni.head(3)

,ANNO,Codice_Nazionale,Codice_Istat,Denominazione Italiana,Provincia,Sigla Provincia,Regione,AreaGeo,CAT_DET_COD_1,CAT_DET_COD_2,...,IND8,IND9,IND10,LQP_COD_1,LQP_COD_2,LQP_COD_3,LQP_COD_4,LQP_COD_5,LQP_COD_6,LQP_COD_S
0,2010,A052,6001,ACQUI TERME,Alessandria,AL,Piemonte,Nord-Ovest,0.007314,0.235773,...,0.0,9.535919,28.0,123.729184,-1.039833,0,0,10.0,5.0,3.4
1,2010,A146,6002,ALBERA LIGURE,Alessandria,AL,Piemonte,Nord-Ovest,0.234971,0.059315,...,0.0,0.000000,3.0,NaN,NaN,1,0,NaN,NaN,NaN
2,2010,A182,6003,ALESSANDRIA,Alessandria,AL,Piemonte,Nord-Ovest,0.001551,0.189163,...,0.0,20.745082,20.0,-51.217919,-12.525939,0,0,3.0,3.0,5.0


In [14]:
print ('SANITY CHECK 2: Numero di comuni dopo la join: ', len(df_info_comuni_fabbisogni))

SANITY CHECK 2: Numero di comuni dopo la join:  6702


* Quindi salviamo il dataset in un csv.

La procedura appena descritta è automatizzatata con la seguente funzione e ripetuta per tutti i file relativi ai fabbisogni.

In [15]:
def HandleData(file_name, comuni_attuali = 'ElencoComuniAttuali_20170918.csv' ,info_comuni = 'info_comuni.txt'):
    """ La funzione crea un nuovo dataset .csv file.
    @file_name: nome del file di fabbisogni
    @comuni_attuali: file ElencoComuniAttuali_20170918.csv che contiene le info ed i codici per i comuni
    @info_comuni: file info_comuni.txt che contiene extra info per i comuni"""
    
    
    # Per ogni anno trovato nel dataset
    for anno in df_fab.ANNO.unique():
        
        # Estrazione righe dataset di interesse
        df_fab_anno = df_fab[df_fab["ANNO"] == anno]
        print ('Numero comuni anno '+ str(anno) +':', len(df_fab_anno['COMUNE_CAT_COD'].unique()))


        # Carico il file 
        df_comuni = pd.read_csv(comuni_attuali, sep = ';')
        # Trattengo le variabili di interesse
        df_comuni = df_comuni[['Codice Nazionale','Codice Istat','Denominazione Italiana', 'Sigla Provincia']] 
        # E le rinomino per poter fare la join con il dataset dei fabbisogni
        df_comuni.columns = ['COMUNE_CAT_COD','ISTAT','Denominazione Italiana', 'Sigla Provincia']


        # Left join tra i fabbisogni ed i dati dei comuni
        df_comuni_cod = pd.merge(df_fab_anno,df_comuni, on = 'COMUNE_CAT_COD',how = "left")

        df_comuni_cod = df_comuni_cod[['COMUNE_CAT_COD','ISTAT', 'Denominazione Italiana',
           'Sigla Provincia','ANNO', 'CAT_DET_COD_1', 'CAT_DET_COD_2',
           'CAT_DET_COD_3', 'CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO',
           'IND1', 'IND3', 'IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10',
           'LQP_COD_1', 'LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5',
           'LQP_COD_6', 'LQP_COD_S']]
        
        print ('SANITY CHECK 1: Numero di comuni dopo la join: ', len(df_comuni_cod))
        
        
        # Carica il file che racchiude informazioni extra sui comuni (provincia, regione, area geografica)
        df_comuni_info = pd.read_csv(info_comuni, sep = ";")
        # Effettua la join con df_comuni_cod 
        df_info_comuni_fabbisogni = pd.merge(df_comuni_cod, df_comuni_info, on = 'ISTAT',how = "left")

        # Rinomina variabili
        df_info_comuni_fabbisogni.columns = ['Codice_Nazionale', 'Codice_Istat', 'Denominazione Italiana', 'Sigla Provincia',
           'ANNO', 'CAT_DET_COD_1', 'CAT_DET_COD_2', 'CAT_DET_COD_3',
           'CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO', 'IND1', 'IND3',
           'IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10', 'LQP_COD_1',
           'LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5', 'LQP_COD_6',
           'LQP_COD_S', 'Comune', 'Provincia', 'SiglaProv', 'Regione', 'AreaGeo',
           'PopResidente', 'PopStraniera', 'DensitaDemografica', 'SuperficieKmq',
           'AltezzaCentro', 'AltezzaMinima', 'AltezzaMassima', 'ZonaAltimetrica',
           'TipoComune', 'GradoUrbaniz', 'IndiceMontanita', 'ZonaClimatica',
           'ZonaSismica', 'ClasseComune', 'Latitudine', 'Longitudine']

        # Seleziona le variabili di interesse
        df_info_comuni_fabbisogni = df_info_comuni_fabbisogni[['ANNO','Codice_Nazionale', 'Codice_Istat',
                                                           'Denominazione Italiana',
                                                           'Provincia','Sigla Provincia',
                                                           'Regione', 'AreaGeo',
                                                           'CAT_DET_COD_1', 'CAT_DET_COD_2', 'CAT_DET_COD_3','CAT_DET_COD_4', 'CAT_DET_COD_5', 'COEFF_TEORICO',
                                                           'IND1', 'IND3','IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10', 
                                                           'LQP_COD_1','LQP_COD_2', 'LQP_COD_3', 'LQP_COD_4', 'LQP_COD_5', 'LQP_COD_6','LQP_COD_S']]

        print ('SANITY CHECK 2: Numero di comuni dopo la join: ', len(df_info_comuni_fabbisogni))
        print ('*'*30)


        df_info_comuni_fabbisogni.to_csv(file_name +str(anno)+ '.csv', sep = ',')

In [16]:
for file_name in lista_file:
    HandleData(file_name)

Numero comuni anno 2010: 6702
SANITY CHECK 1: Numero di comuni dopo la join:  6702
SANITY CHECK 2: Numero di comuni dopo la join:  6702
******************************
Numero comuni anno 2013: 6699
SANITY CHECK 1: Numero di comuni dopo la join:  6699
SANITY CHECK 2: Numero di comuni dopo la join:  6699
******************************
Numero comuni anno 2010: 6702
SANITY CHECK 1: Numero di comuni dopo la join:  6702
SANITY CHECK 2: Numero di comuni dopo la join:  6702
******************************
Numero comuni anno 2013: 6699
SANITY CHECK 1: Numero di comuni dopo la join:  6699
SANITY CHECK 2: Numero di comuni dopo la join:  6699
******************************
Numero comuni anno 2010: 6702
SANITY CHECK 1: Numero di comuni dopo la join:  6702
SANITY CHECK 2: Numero di comuni dopo la join:  6702
******************************
Numero comuni anno 2013: 6699
SANITY CHECK 1: Numero di comuni dopo la join:  6699
SANITY CHECK 2: Numero di comuni dopo la join:  6699
*****************************